In [1]:
import pytoon

def rasterize(rgba_Dt, xdim=(-1,1,20), ydim=(-1,1,20), pixel_aspect_ratio=1, width=100):
    # rgba_Dt should be a function of x and y that returns a 2-tuple of rgbs and Dt, where
    # where Dt is the time resolution to use at that point in space and
    # rgba is a function of time (and it can ignore the time argument), returning a 2-tuple
    # which is the rgb code (in #RRGGBB form) and alpha value at that x,y point (at that time);
    # the code can recover if the alpha value is omitted (a value instead of a tuple is returned)
    # by omitting the alpha channel (no transparency).
    #
    Py = pixel_aspect_ratio
    Px = 1 / Py            # hard-coded:  pixel area is 1
    px = (1.001) * Px/2    # fudge factor to overlap edges just slightly ...
    py = (1.001) * Py/2    # ... looks a lot better than leaving even a tiny gap (due to roundings?)
    pixel = pytoon.polygon(points=[(-px,-py),(px,-py),(px,py),(-px,py)], lstyle=False)
    #
    def color(x, y, layer):
        rgba, Dt = rgba_Dt(x,y)
        if layer=="upper":
            alpha = lambda a: a/2
        else:
            alpha = lambda a: a/(2-a)
        def value(_t_):
            rgb_a = rgba(_t_)
            try:
                rgb, a = rgb_a
            except TypeError:
                rgb, a = rgb_a, 1
            a = 1 if (a is None) else a
            return pytoon.colordef(rgb=rgb, a=alpha(a))
        return pytoon.animated(value, Dt=Dt)
    #
    xmin, xmax, Nx = xdim
    ymin, ymax, Ny = ydim
    Dx = (xmax-xmin) / Nx
    Dy = (ymax-ymin) / Ny
    lower = []
    for i in range(Nx+1):
        x = xmin + i*Dx
        for j in range(Ny+1):
            y = ymin + j*Dy
            lower += [ pixel(fstyle=color(x,y,"upper")).T(i*Px, j*Py) ]
    upper = []
    for i in range(Nx):
        x = xmin + (i+1/2)*Dx 
        for j in range(Ny):
            y = ymin + (j+1/2)*Dy
            upper += [pixel(fstyle=color(x,y,"upper")).T((i+1/2)*Px, (j+1/2)*Py)]
    #
    return pytoon.composite([*lower, *upper]).S(width/(Px*Nx))

In [2]:
from math  import sqrt, cos, sin, pi, atan2
from cmath import polar, exp
from util  import int_round

In [3]:
def color_map(phi):
    red    = 0xcc, 0x33, 0x33
    orange = 0xcc, 0x55, 0
    yellow = 0x99, 0x88, 0
    green  = 0,    0x77, 0x11
    blue   = 0x66, 0x77, 0xff
    purple = 0xbb, 0x11, 0xee
    phi /= pi
    if   phi<-2/3:
        w = 3*(phi + 1)
        v = 1 - w
        r1, g1, b1 = red
        r2, g2, b2 = orange
        rr = int_round(v*r1 + w*r2)
        gg = int_round(v*g1 + w*g2)
        bb = int_round(v*b1 + w*b2)
    elif phi<-1/3:
        w = 3*(phi + 2/3)
        v = 1 - w
        r1, g1, b1 = orange
        r2, g2, b2 = yellow
        rr = int_round(v*r1 + w*r2)
        gg = int_round(v*g1 + w*g2)
        bb = int_round(v*b1 + w*b2)
    elif phi< 0:
        w = 3*(phi + 1/3)
        v = 1 - w
        r1, g1, b1 = yellow
        r2, g2, b2 = green
        rr = int_round(v*r1 + w*r2)
        gg = int_round(v*g1 + w*g2)
        bb = int_round(v*b1 + w*b2)
    elif phi< 1/3:
        w = 3*phi
        v = 1 - w
        r1, g1, b1 = green
        r2, g2, b2 = blue
        rr = int_round(v*r1 + w*r2)
        gg = int_round(v*g1 + w*g2)
        bb = int_round(v*b1 + w*b2)
    elif phi< 2/3:
        w = 3*(phi - 1/3)
        v = 1 - w
        r1, g1, b1 = blue
        r2, g2, b2 = purple
        rr = int_round(v*r1 + w*r2)
        gg = int_round(v*g1 + w*g2)
        bb = int_round(v*b1 + w*b2)
    else:
        w = 3*(phi - 2/3)
        v = 1 - w
        r1, g1, b1 = purple
        r2, g2, b2 = red
        rr = int_round(v*r1 + w*r2)
        gg = int_round(v*g1 + w*g2)
        bb = int_round(v*b1 + w*b2)
    return "#{:02x}{:02x}{:02x}".format(rr,gg,bb)

In [4]:
I = 1j
T = 20
L = 1.6
R = 35
S = 20

def s(x,y,t):
    return exp(-(x**2 + y**2))

def p(x,y,t):
    return 2 * y * exp(-(x**2 + y**2))

def s_p(x,y,t):
    v = 1/T
    if t<0:
        a, b = 1, 0
    else:
        a, b = cos(2*pi*v*t), sin(2*pi*v*t)
    return a*exp(I*2*pi*2*t)*s(x,y,t) + b*exp(I*2*pi*t/2)*p(x,y,t)

def orbital(f):
    def F(x,y):
        d = sqrt(x**2 + y**2)
        Dt = 0.1
        #if d>1.2:  Dt *= (d-0.2)**2
        def rgba(t):
            rad, phi = polar(f(x,y,t))
            a = 1.4 * rad**2
            rgb = color_map(phi)
            return rgb, a
        return rgba, Dt
    return F

def field(_t_):
    a = exp(-(_t_**2)*2).real if _t_<0 else 1
    return 15, 50-a*25*cos(2*pi*3*_t_/2)

def left_arr(_t_):
    a = exp(-(_t_**2)*2).real if _t_<0 else 1
    return -5, a*5*cos(2*pi*3*_t_/2)
        
def right_arr(_t_):
    a = exp(-(_t_**2)*2).real if _t_<0 else 1
    return 5, a*5*cos(2*pi*3*_t_/2)

proton = pytoon.composite([
    pytoon.circle(radius=3, fstyle="#555555", lstyle=False),
    pytoon.line(begin=(0,-2), end=(0,2)),
    pytoon.line(begin=(-2,0), end=(2,0))    
])

pytoon.composite([
    proton.T(50,50),
    pytoon.line(begin=(15,50), end=pytoon.animated(field,Dt=0.025), lstyle="#999999"),
    pytoon.line(begin=pytoon.animated(field,Dt=0.025), displacement=pytoon.animated(left_arr,Dt=0.025), lstyle="#999999"),
    pytoon.line(begin=pytoon.animated(field,Dt=0.025), displacement=pytoon.animated(right_arr,Dt=0.025), lstyle="#999999"),
    rasterize(orbital(s_p), xdim=(-L,L,R), ydim=(-L,L,R))
]).svg("outputs/rasterized", time=(-2,T/3.8), duration=3*T, background="black", controls=pytoon.struct(scale=0.25, location=(20,0)))

[link to animation file](files/outputs/rasterized.svg)